**Group-1**

**Title:** Classification of Doremon cartoon characters from a custom Image dataset using a CNN model.

**Team members:**
Santosh Reddy Edulapalle - A20501739
Venkata Siva Rupesh Akurati - A20501754
Jack Harrison Mohr -A20503445

**This is our .ipynb file for the project. We are going to code everything in this file.**

**Image dataset:** We are using OpenCV to read the video and capture image with rate of 1 frame per second.

**ImageSS** This function takes in path of the video and uses OpenCV to read each frame and captures the images.

In [18]:
#importing cv2 from OpenCV
import cv2
from datetime import datetime

def ImageSS(videoPath):

    videoObject = cv2.VideoCapture(videoPath)
    frameNumber = 1

    # checks whether frames were extracted
    boolean = 1
    start = datetime.now()
    try:
        while boolean:
            boolean, image = videoObject.read()
            cv2.imwrite("frame%d.jpg" %frameNumber, image)
            frameNumber += 1
    #exception handling
    except:
        print("All the frames are read!")
    finally:
        end = datetime.now()
    #calculating execution time.
    executionTime = (end - start).total_seconds() * 10**3
    print(f"The execution time of above program is : {executionTime:.03f}ms")


The below cell is commented to prevent it from accidental running.Uncomment it to use the program to capture video frames.

In [19]:
#ImageSS("/Users/santosh/Desktop/Neeraja Bday Oct 14.mp4")